# Getting Spotify data for the album 'Little Dark Age'
### Code borrowed from Dr. David Silva's [repo](https://github.com/dataesilva/22110-FA21) from the EMAT 22110 Class 

Importing necessary packages

In [42]:
import pandas as pd
import requests
import base64
import six
import json
from matplotlib import pyplot as plt

Reading in tokens to access the Spotify API

In [2]:
tokens = pd.read_csv('spotifyAPI.txt', sep = '\t', header = None).T
tokens.columns = tokens.iloc[0]

Formatting tokens variable to be readable for the functions

In [3]:
tokens = tokens [1:]

Creates header to connect to the client server with token information

In [5]:
def start_session_headers(client_id, client_secret):
    client_cred = base64.b64encode(str(client_id + ":" + client_secret).encode('ascii'))
    header = {'Authorization': 'Basic %s' % (client_cred.decode('ascii'))}
    return header

Connects to the client on the Spotify server to get information

In [6]:
def connect_to_client_cred(url, header):
        payload = {'grant_type' : 'client_credentials'}
        response = requests.request("POST", url, headers = header, data = payload )
        print(response.status_code)
        if response.status_code != 200:
            raise Exception(response.status_code, response.text)
        return response.json()

Gets access token from the client to be able to call the API

In [7]:
def get_access_token(client_id, client_secret):
    url = "https://accounts.spotify.com/api/token"
    header = start_session_headers(client_id, client_secret)
    json_response = connect_to_client_cred(url, header)
    return json_response

Creates 'session_token' variable to actually call the API in below functions

In [8]:
session_token = get_access_token(tokens['clientID'].iloc[0], tokens['clientSecret'].iloc[0])

200


Creates new header to call the API

In [13]:
def create_header(session_token):
    headers = {"Authorization": "Bearer {}".format(session_token)}
    return headers

Connecting to the API

In [14]:
def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers = headers)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

Function that gets information on an album available on Spotify through the URI variable

In [15]:
def get_an_album(album_id):
    url = 'https://api.spotify.com/v1/albums/{}'.format(album_id)
    header = create_header(session_token['access_token'])
    json_response = connect_to_endpoint(url, header)
    return json_response

Function that gets information about a track available on Spotify through the URI variable

In [16]:
def get_a_track(track_id):
    url = 'https://api.spotify.com/v1/tracks/{}'.format(track_id)
    header = create_header(session_token['access_token'])
    json_response = connect_to_endpoint(url, header)
    return json_response

Calls all the above functions to be able to get information on the album Little Dark Age

In [17]:
littleDarkAge = get_an_album('7GjVWG39IOj4viyWplJV4H')

200


Looks at the keys from the json in the API call response to determine how to make the Data Frame

In [18]:
littleDarkAge.keys()

dict_keys(['album_type', 'artists', 'available_markets', 'copyrights', 'external_ids', 'external_urls', 'genres', 'href', 'id', 'images', 'label', 'name', 'popularity', 'release_date', 'release_date_precision', 'total_tracks', 'tracks', 'type', 'uri'])

Gets data on each individual track from the Little Dark Age album

In [28]:
trackData = littleDarkAge['tracks']

Creates a Data Frame with all of the information about the tracks from the album API call

In [33]:
trackDataDf = pd.DataFrame(trackData['items'])

Creates a Data Frame with only the track name and track URI

In [37]:
songNames = trackDataDf[['name', 'uri']]

In [38]:
songNames

,name,uri
0,She Works Out Too Much,spotify:track:3XOKU8CKSiQsuQHD5vhzo5
1,Little Dark Age,spotify:track:2Y0iGXY6m6immVb2ktbseM
2,When You Die,spotify:track:3td69vL9Py7Ai9wfXYnvji
3,Me and Michael,spotify:track:0t4z0WaQomQqPONghWn8c2
4,TSLAMP,spotify:track:43rJQjd6zfyTP58TNI9JAi
5,James,spotify:track:54hdBH6z6hMBUpC9fAMs3s
6,Days That Got Away,spotify:track:5J2SAAtyfFXOsctaYiUuLO
7,One Thing Left to Try,spotify:track:52pWggDlN5mVeX111i06PP
8,When You're Small,spotify:track:0fokoG1PTmfQr6HrBoKzo1
9,Hand It Over,spotify:track:1s2oewEkFG9eF9Mq7MyvCV


Calls the Track API to get information on each track in the album

In [23]:
def track_audio_features(id):
    url = 'https://api.spotify.com/v1/audio-features/{}'.format(id)
    header = create_header(session_token['access_token'])
    json_response = connect_to_endpoint(url, header)
    return json_response

Calling the track_audio_features function on each track ID in the album

In [24]:
songInfo = [track_audio_features(x['id']) for x in littleDarkAge['tracks']['items']]

200
200
200
200
200
200
200
200
200
200


Creating a Data Frame based on the returned information on each track

In [26]:
songInfoDf = pd.DataFrame(songInfo)

In [27]:
songInfoDf

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.633,0.919,5,-6.208,0,0.0343,0.00596,0.273000,0.430,0.805,147.970,audio_features,3XOKU8CKSiQsuQHD5vhzo5,spotify:track:3XOKU8CKSiQsuQHD5vhzo5,https://api.spotify.com/v1/tracks/3XOKU8CKSiQs...,https://api.spotify.com/v1/audio-analysis/3XOK...,278387,4
1,0.705,0.712,6,-6.156,1,0.0385,0.01020,0.000855,0.100,0.620,97.512,audio_features,2Y0iGXY6m6immVb2ktbseM,spotify:track:2Y0iGXY6m6immVb2ktbseM,https://api.spotify.com/v1/tracks/2Y0iGXY6m6im...,https://api.spotify.com/v1/audio-analysis/2Y0i...,299960,4
2,0.645,0.938,11,-4.557,0,0.0391,0.09460,0.022600,0.159,0.495,140.886,audio_features,3td69vL9Py7Ai9wfXYnvji,spotify:track:3td69vL9Py7Ai9wfXYnvji,https://api.spotify.com/v1/tracks/3td69vL9Py7A...,https://api.spotify.com/v1/audio-analysis/3td6...,263880,4
3,0.628,0.841,9,-4.583,1,0.0268,0.02850,0.003320,0.254,0.706,111.985,audio_features,0t4z0WaQomQqPONghWn8c2,spotify:track:0t4z0WaQomQqPONghWn8c2,https://api.spotify.com/v1/tracks/0t4z0WaQomQq...,https://api.spotify.com/v1/audio-analysis/0t4z...,289853,4
4,0.602,0.879,9,-5.283,0,0.0994,0.22000,0.000236,0.107,0.733,174.110,audio_features,43rJQjd6zfyTP58TNI9JAi,spotify:track:43rJQjd6zfyTP58TNI9JAi,https://api.spotify.com/v1/tracks/43rJQjd6zfyT...,https://api.spotify.com/v1/audio-analysis/43rJ...,270013,4
5,0.515,0.898,8,-6.682,1,0.0291,0.12300,0.380000,0.199,0.778,113.983,audio_features,54hdBH6z6hMBUpC9fAMs3s,spotify:track:54hdBH6z6hMBUpC9fAMs3s,https://api.spotify.com/v1/tracks/54hdBH6z6hMB...,https://api.spotify.com/v1/audio-analysis/54hd...,232173,4
6,0.667,0.782,7,-7.681,0,0.0318,0.05120,0.896000,0.645,0.886,132.983,audio_features,5J2SAAtyfFXOsctaYiUuLO,spotify:track:5J2SAAtyfFXOsctaYiUuLO,https://api.spotify.com/v1/tracks/5J2SAAtyfFXO...,https://api.spotify.com/v1/audio-analysis/5J2S...,284547,3
7,0.576,0.940,8,-4.541,1,0.0445,0.00678,0.000660,0.308,0.667,128.017,audio_features,52pWggDlN5mVeX111i06PP,spotify:track:52pWggDlN5mVeX111i06PP,https://api.spotify.com/v1/tracks/52pWggDlN5mV...,https://api.spotify.com/v1/audio-analysis/52pW...,260347,4
8,0.465,0.465,2,-7.792,1,0.0276,0.61200,0.006390,0.223,0.242,140.001,audio_features,0fokoG1PTmfQr6HrBoKzo1,spotify:track:0fokoG1PTmfQr6HrBoKzo1,https://api.spotify.com/v1/tracks/0fokoG1PTmfQ...,https://api.spotify.com/v1/audio-analysis/0fok...,210880,4
9,0.378,0.614,1,-8.941,0,0.0387,0.70600,0.007660,0.129,0.481,175.926,audio_features,1s2oewEkFG9eF9Mq7MyvCV,spotify:track:1s2oewEkFG9eF9Mq7MyvCV,https://api.spotify.com/v1/tracks/1s2oewEkFG9e...,https://api.spotify.com/v1/audio-analysis/1s2o...,253360,4


Adding a column for the name of the track to connect the track name to the other information available

In [39]:
songInfoNamesDf = pd.merge(songInfoDf, songNames, left_on = "uri", right_on = 'uri', how = 'left').drop("uri", axis = 1)

In [40]:
songInfoNamesDf

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,name
0,0.633,0.919,5,-6.208,0,0.0343,0.00596,0.273000,0.430,0.805,147.970,audio_features,3XOKU8CKSiQsuQHD5vhzo5,https://api.spotify.com/v1/tracks/3XOKU8CKSiQs...,https://api.spotify.com/v1/audio-analysis/3XOK...,278387,4,She Works Out Too Much
1,0.705,0.712,6,-6.156,1,0.0385,0.01020,0.000855,0.100,0.620,97.512,audio_features,2Y0iGXY6m6immVb2ktbseM,https://api.spotify.com/v1/tracks/2Y0iGXY6m6im...,https://api.spotify.com/v1/audio-analysis/2Y0i...,299960,4,Little Dark Age
2,0.645,0.938,11,-4.557,0,0.0391,0.09460,0.022600,0.159,0.495,140.886,audio_features,3td69vL9Py7Ai9wfXYnvji,https://api.spotify.com/v1/tracks/3td69vL9Py7A...,https://api.spotify.com/v1/audio-analysis/3td6...,263880,4,When You Die
3,0.628,0.841,9,-4.583,1,0.0268,0.02850,0.003320,0.254,0.706,111.985,audio_features,0t4z0WaQomQqPONghWn8c2,https://api.spotify.com/v1/tracks/0t4z0WaQomQq...,https://api.spotify.com/v1/audio-analysis/0t4z...,289853,4,Me and Michael
4,0.602,0.879,9,-5.283,0,0.0994,0.22000,0.000236,0.107,0.733,174.110,audio_features,43rJQjd6zfyTP58TNI9JAi,https://api.spotify.com/v1/tracks/43rJQjd6zfyT...,https://api.spotify.com/v1/audio-analysis/43rJ...,270013,4,TSLAMP
5,0.515,0.898,8,-6.682,1,0.0291,0.12300,0.380000,0.199,0.778,113.983,audio_features,54hdBH6z6hMBUpC9fAMs3s,https://api.spotify.com/v1/tracks/54hdBH6z6hMB...,https://api.spotify.com/v1/audio-analysis/54hd...,232173,4,James
6,0.667,0.782,7,-7.681,0,0.0318,0.05120,0.896000,0.645,0.886,132.983,audio_features,5J2SAAtyfFXOsctaYiUuLO,https://api.spotify.com/v1/tracks/5J2SAAtyfFXO...,https://api.spotify.com/v1/audio-analysis/5J2S...,284547,3,Days That Got Away
7,0.576,0.940,8,-4.541,1,0.0445,0.00678,0.000660,0.308,0.667,128.017,audio_features,52pWggDlN5mVeX111i06PP,https://api.spotify.com/v1/tracks/52pWggDlN5mV...,https://api.spotify.com/v1/audio-analysis/52pW...,260347,4,One Thing Left to Try
8,0.465,0.465,2,-7.792,1,0.0276,0.61200,0.006390,0.223,0.242,140.001,audio_features,0fokoG1PTmfQr6HrBoKzo1,https://api.spotify.com/v1/tracks/0fokoG1PTmfQ...,https://api.spotify.com/v1/audio-analysis/0fok...,210880,4,When You're Small
9,0.378,0.614,1,-8.941,0,0.0387,0.70600,0.007660,0.129,0.481,175.926,audio_features,1s2oewEkFG9eF9Mq7MyvCV,https://api.spotify.com/v1/tracks/1s2oewEkFG9e...,https://api.spotify.com/v1/audio-analysis/1s2o...,253360,4,Hand It Over


Exporting the Data Frame to use in the lesson!

In [46]:
songInfoNamesDf.to_csv(r"C:\Users\kathm\EMAT22110\spotifyCSVData.csv")